<a href="https://colab.research.google.com/github/Osondu-ifunanya/Ecosystem-service-valuation-using-ML-and-spatial-metrics/blob/main/Ecosystem%20service%20valuation%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
rainfall_runoff_lstm.py

Rainfall-runoff modeling with LSTM using synthetic basin-level inputs.

Produces:
 - synthetic time series for multiple basins
 - LSTM model training and evaluation
 - RMSE and Nash-Sutcliffe Efficiency (NSE)
 - plots of observed vs predicted streamflow
 - exports predictions to Excel

Requires:
    pip install numpy pandas matplotlib scikit-learn tensorflow openpyxl
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import os

# ---------------------------
# Utility: Nash-Sutcliffe Efficiency
# ---------------------------
def nse(observed, simulated):
    obs = np.array(observed)
    sim = np.array(simulated)
    denom = np.sum((obs - obs.mean())**2)
    if denom == 0:
        return np.nan
    return 1 - np.sum((sim - obs)**2) / denom

# ---------------------------
# 1) Generate synthetic basin-level time series
# ---------------------------
def generate_synthetic_basin_series(n_days=365*4, n_basins=5, seed=42):
    """
    Generate synthetic daily timeseries for multiple basins.
    Features: rainfall (mm), temperature (C), antecedent soil moisture (0-1)
    Target: streamflow (m3/s) — synthetic function of rainfall and antecedent moisture
    """
    rng = np.random.default_rng(seed)
    dates = pd.date_range("2015-01-01", periods=n_days, freq="D")

    all_basins = {}
    for b in range(n_basins):
        # seasonal rainfall pattern (sinusoid) + random storms
        seasonal = 20 + 15 * np.sin(2 * np.pi * (np.arange(n_days) % 365) / 365 + rng.uniform(0, 2*np.pi))
        rainfall = np.maximum(0, seasonal * 0.2 * rng.normal(1.0, 0.3, n_days) + rng.exponential(3.0, n_days))
        # reduce rainfall in dry basins somewhat
        rainfall *= rng.uniform(0.6, 1.4)

        # temperature seasonal
        temp = 10 + 8 * np.sin(2 * np.pi * (np.arange(n_days) % 365) / 365 + rng.uniform(0, 2*np.pi)) + rng.normal(0, 1.5, n_days)

        # antecedent soil moisture evolves with rainfall and evapotranspiration
        soil = np.zeros(n_days)
        soil[0] = rng.uniform(0.2, 0.6)
        for t in range(1, n_days):
            recharge = 0.02 * rainfall[t-1]  # small fraction
            evap = 0.01 * max(0, temp[t-1]-5)
            soil[t] = np.clip(soil[t-1] + recharge - evap + rng.normal(0, 0.005), 0, 1)

        # true streamflow: nonlinear function: baseflow + quickflow from recent storms modulated by soil moisture
        baseflow = 0.5 + 0.5 * rng.uniform(0.5, 1.5)  # basin-specific base
        runoff = np.zeros(n_days)
        for t in range(n_days):
            # quick response from recent rainfall (last 3 days)
            recent_rain = rainfall[max(0, t-3):t+1].sum()
            quick = 0.1 * recent_rain * (0.5 + soil[t])  # wetter soils lead to higher quickflow here (synthetic choice)
            seasonal_factor = 1 + 0.3 * np.sin(2*np.pi*(t%365)/365)
            runoff[t] = max(0, baseflow * seasonal_factor + quick + rng.normal(0, baseflow*0.2))

        df = pd.DataFrame({
            "date": dates,
            "rainfall_mm": rainfall,
            "temperature_C": temp,
            "soil_moisture": soil,
            "streamflow_m3s": runoff
        })
        all_basins[f"basin_{b+1:02d}"] = df

    return all_basins

# ---------------------------
# 2) Prepare sequences for LSTM
# ---------------------------
def create_sequences(df, input_cols, target_col, seq_len=14, horizon=1):
    """
    Convert a DataFrame for one basin into sequences for LSTM.
    - seq_len: number of past days used as input
    - horizon: days ahead to predict (1 -> next day)
    Returns X (n_samples, seq_len, n_features), y (n_samples,)
    """
    data = df[input_cols].values
    target = df[target_col].values
    n = len(df)
    X, y = [], []
    for i in range(seq_len, n - horizon + 1):
        X.append(data[i-seq_len:i, :])
        y.append(target[i + horizon - 1])  # predict runoff at time i+horizon-1
    return np.array(X), np.array(y)

# ---------------------------
# 3) Build and train LSTM model
# ---------------------------
def build_lstm_model(n_timesteps, n_features, n_units=64, dropout=0.2):
    model = models.Sequential([
        layers.Input(shape=(n_timesteps, n_features)),
        layers.LSTM(n_units, return_sequences=True),
        layers.Dropout(dropout),
        layers.LSTM(n_units//2, return_sequences=False),
        layers.Dropout(dropout*0.5),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# ---------------------------
# 4) Main pipeline
# ---------------------------
if __name__ == "__main__":
    # settings
    N_BASINS = 6
    N_DAYS = 365 * 3  # 3 years daily
    SEQ_LEN = 21      # use 21 days of history
    HORIZON = 1
    TEST_FRACTION = 0.2
    RANDOM_SEED = 42

    # generate synthetic data
    basins = generate_synthetic_basin_series(n_days=N_DAYS, n_basins=N_BASINS, seed=RANDOM_SEED)
    print(f"Generated synthetic data for {len(basins)} basins, {N_DAYS} days each.")

    # concatenate basins into one dataset for training (multi-site approach)
    input_cols = ["rainfall_mm", "temperature_C", "soil_moisture"]
    target_col = "streamflow_m3s"

    X_list, y_list, meta = [], [], []
    for basin_name, df in basins.items():
        Xb, yb = create_sequences(df, input_cols, target_col, seq_len=SEQ_LEN, horizon=HORIZON)
        # keep dates aligned for exports later
        dates = df['date'].values[SEQ_LEN - 1: len(df) - HORIZON + 1]
        # append
        X_list.append(Xb)
        y_list.append(yb)
        meta.append(pd.DataFrame({
            "basin": basin_name,
            "date": dates[:len(yb)]
        }))

    X_all = np.vstack(X_list)
    y_all = np.hstack(y_list)
    meta_all = pd.concat(meta, ignore_index=True)
    print("Total samples (pooled):", X_all.shape[0])

    # Split into training and test by time-block per-basin to avoid leakage:
    # We'll split meta_all sequentially per-basin: simpler approach: use first (1-test_frac) portion for train per basin
    # Build boolean mask for train/test samples
    train_mask = np.zeros(len(meta_all), dtype=bool)
    idx = 0
    for basin_name, df in basins.items():
        n_samples_basin = len(df) - SEQ_LEN - (HORIZON - 1)
        n_train = int(np.floor((1 - TEST_FRACTION) * n_samples_basin))
        train_mask[idx: idx + n_train] = True
        idx += n_samples_basin
    test_mask = ~train_mask

    X_train = X_all[train_mask]
    y_train = y_all[train_mask]
    X_test = X_all[test_mask]
    y_test = y_all[test_mask]
    meta_test = meta_all[test_mask].reset_index(drop=True)

    print("Train samples:", X_train.shape[0], "Test samples:", X_test.shape[0])

    # Scale features (fit scaler on training data only)
    n_features = X_all.shape[2]
    scaler = StandardScaler()
    # reshape to 2D for scaler
    X_train_2d = X_train.reshape(-1, n_features)
    X_test_2d = X_test.reshape(-1, n_features)
    scaler.fit(X_train_2d)
    X_train_s = scaler.transform(X_train_2d).reshape(X_train.shape)
    X_test_s = scaler.transform(X_test_2d).reshape(X_test.shape)

    # Scale target? We'll predict streamflow in original units; optionally log-transform
    # use log1p transform to stabilize variance
    y_train_log = np.log1p(y_train)
    y_test_log = np.log1p(y_test)

    # Build model
    model = build_lstm_model(n_timesteps=SEQ_LEN, n_features=n_features, n_units=128, dropout=0.2)
    model.summary()

    # Callbacks
    cb = [
        callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
        callbacks.ModelCheckpoint("best_lstm_runoff.h5", save_best_only=True, monitor='val_loss')
    ]

    # Train (use a validation split from training set)
    history = model.fit(
        X_train_s, y_train_log,
        validation_split=0.15,
        epochs=80,
        batch_size=64,
        callbacks=cb,
        verbose=2
    )

    # Predict on test (log-space); invert transform
    y_pred_log = model.predict(X_test_s).ravel()
    y_pred = np.expm1(y_pred_log)
    # clip negatives
    y_pred = np.clip(y_pred, 0, None)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    nse_val = nse(y_test, y_pred)
    print(f"\nTest RMSE: {rmse:.3f} m3/s")
    print(f"Test NSE: {nse_val:.3f}")

    # Plot time series for first few basins in test set for visual check
    # pick first 3 basins represented in test meta
    sample_basins = meta_test['basin'].unique()[:3]
    plt.figure(figsize=(12, 8))
    for i, basin_name in enumerate(sample_basins, 1):
        mask_b = meta_test['basin'] == basin_name
        dates_b = pd.to_datetime(meta_test.loc[mask_b, 'date'])
        obs_b = y_test[mask_b]
        pred_b = y_pred[mask_b]
        plt.subplot(len(sample_basins), 1, i)
        plt.plot(dates_b, obs_b, label='Observed', linewidth=1)
        plt.plot(dates_b, pred_b, label='Predicted', linewidth=1)
        plt.title(basin_name)
        plt.ylabel("Streamflow (m3/s)")
        plt.legend()
    plt.tight_layout()
    plt.show()

    # Scatter observed vs predicted
    plt.figure(figsize=(5,5))
    plt.scatter(y_test, y_pred, alpha=0.4, s=10)
    m = max(y_test.max(), y_pred.max())
    plt.plot([0, m], [0, m], 'r--')
    plt.xlabel("Observed (m3/s)")
    plt.ylabel("Predicted (m3/s)")
    plt.title(f"Observed vs Predicted (RMSE={rmse:.2f}, NSE={nse_val:.2f})")
    plt.grid(True)
    plt.show()

    # Export test predictions to Excel
    out_df = meta_test.copy()
    out_df['observed_streamflow_m3s'] = y_test
    out_df['predicted_streamflow_m3s'] = y_pred
    out_path = "lstm_runoff_predictions.xlsx"
    out_df.to_excel(out_path, index=False)
    print(f"Exported test predictions to: {out_path}")

    # Save training history plot
    plt.figure()
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.yscale('log')
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE, log scale)")
    plt.legend()
    plt.title("Training history")
    plt.tight_layout()
    plt.savefig("training_history_runoff.png", dpi=150)
    print("Saved training history plot: training_history_runoff.png")

    print("Done.")
